In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Load the Wine dataset
df=pd.read_csv("C:\\Users\\akhil\\Downloads\\Global-Superstore.csv",nrows=500)

In [3]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,7/31/2012,7/31/2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,02-05-2013,02-07-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,10/17/2013,10/18/2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,1/28/2013,1/30/2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,11-05-2013,11-06-2013,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [4]:
df.isnull().sum()

Row ID              0
Order ID            0
Order Date          0
Ship Date           0
Ship Mode           0
Customer ID         0
Customer Name       0
Segment             0
City                0
State               0
Country             0
Postal Code       400
Market              0
Region              0
Product ID          0
Category            0
Sub-Category        0
Product Name        0
Sales               0
Quantity            0
Discount            0
Profit              0
Shipping Cost       0
Order Priority      0
dtype: int64

In [5]:
df.dtypes

Row ID              int64
Order ID           object
Order Date         object
Ship Date          object
Ship Mode          object
Customer ID        object
Customer Name      object
Segment            object
City               object
State              object
Country            object
Postal Code       float64
Market             object
Region             object
Product ID         object
Category           object
Sub-Category       object
Product Name       object
Sales             float64
Quantity            int64
Discount          float64
Profit            float64
Shipping Cost     float64
Order Priority     object
dtype: object

In [6]:
df.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit,Shipping Cost
count,500.0000,100.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,25521.8560,47647.580000,2567.003479,6.300000,0.083604,480.515932,435.223480
std,12555.9382,33513.528607,1587.321484,2.654825,0.132907,924.977352,139.646516
min,58.0000,2149.000000,585.456000,1.000000,0.000000,-6599.978000,290.690000
25%,16674.0000,18844.500000,1525.806000,4.000000,0.000000,121.447500,328.247500
50%,25441.5000,42737.500000,2284.707000,6.000000,0.000000,378.060000,384.045000
75%,33878.2500,90008.000000,3089.272000,8.000000,0.100000,757.076250,498.415000
max,51284.0000,98226.000000,17499.950000,14.000000,0.700000,8399.976000,933.570000


In [7]:
df.drop(columns=['Postal Code'],inplace=True)

In [8]:
df_num = df.select_dtypes(include = [np.number])
df_cat = df.select_dtypes(include = ['object'])

print(df_num.columns)
print("---------------------------------------------------------------------")
print(df_cat.columns)

Index(['Row ID', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost'], dtype='object')
---------------------------------------------------------------------
Index(['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'Segment', 'City', 'State', 'Country', 'Market',
       'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name',
       'Order Priority'],
      dtype='object')


In [9]:
df_num_corr=df_num.corr()

In [10]:
df_num_corr["Profit"]

Row ID           0.086334
Sales            0.537254
Quantity         0.062135
Discount        -0.524926
Profit           1.000000
Shipping Cost    0.093681
Name: Profit, dtype: float64

In [11]:
df_num_list = []
df_num_list.extend(df_num_corr[(df_num_corr['Profit']> 0.3)].index.values)
df_num_list.extend(df_num_corr[(df_num_corr['Profit']<- 0.3)].index.values)

In [12]:
df_num_list

['Sales', 'Profit', 'Discount']

In [13]:
from scipy.stats import f_oneway

In [14]:
df_cat['pf']=df_num['Profit']

In [15]:
groups = [df_cat['pf'][df_cat['Region'] == category] for category in df_cat['Region'].unique()]

# Perform ANOVA
f_stat, p_value = f_oneway(*groups)
print(f"F-statistic: {f_stat}, P-value: {p_value}")

F-statistic: 0.7100089500040777, P-value: 0.7422301695099094


In [16]:
#This will store the names of categorical columns that are significantly associated with 'sp'.
c_list=[]
#This will store the names of columns that are not significantly associated with 'sp'.
nc_list=[]
for c1 in list(df_cat.columns):
    if c1=='pf':
        continue
    else:
        groups=[df_cat['pf'][df_cat[c1] == category] for category in df_cat[c1].unique()]
        f_stat, p_value = f_oneway(*groups)
        print(f"column : {c1}, F-statistic: {f_stat}, P-value: {p_value}")
        if p_value < 0.05:
            c_list.append(c1)
        else:
            nc_list.append(c1)


column : Order ID, F-statistic: 7.65700166380363, P-value: 5.1018025640478576e-05
column : Order Date, F-statistic: 1.9466778711345682, P-value: 4.028041229177093e-05
column : Ship Date, F-statistic: 1.6737592408399473, P-value: 0.000998472848805729
column : Ship Mode, F-statistic: 10.559396723884987, P-value: 9.64947321307555e-07
column : Customer ID, F-statistic: 1.377845381348465, P-value: 0.024171214804740205
column : Customer Name, F-statistic: 1.5443945613673205, P-value: 0.0021659229156707287
column : Segment, F-statistic: 0.6693806293074301, P-value: 0.5124866101315592
column : City, F-statistic: 2.189959583221864, P-value: 3.368586306757815e-08
column : State, F-statistic: 1.675376829347728, P-value: 2.3792562184258793e-05
column : Country, F-statistic: 1.2558977160992595, P-value: 0.09357010069544935
column : Market, F-statistic: 1.429929108983939, P-value: 0.20113640381099662
column : Region, F-statistic: 0.7100089500040777, P-value: 0.7422301695099094
column : Product ID, F

In [17]:
c_list

['Order ID',
 'Order Date',
 'Ship Date',
 'Ship Mode',
 'Customer ID',
 'Customer Name',
 'City',
 'State',
 'Category',
 'Sub-Category',
 'Product Name',
 'Order Priority']

In [18]:
nc_list

['Segment', 'Country', 'Market', 'Region', 'Product ID']

In [19]:
df_cat1=df_cat[c_list]

In [20]:
df_cat1.columns

Index(['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'City', 'State', 'Category', 'Sub-Category',
       'Product Name', 'Order Priority'],
      dtype='object')

In [21]:
df_cat1.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,City,State,Category,Sub-Category,Product Name,Order Priority
0,CA-2012-124891,7/31/2012,7/31/2012,Same Day,RH-19495,Rick Hansen,New York City,New York,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,Critical
1,IN-2013-77878,02-05-2013,02-07-2013,Second Class,JR-16210,Justin Ritter,Wollongong,New South Wales,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",Critical
2,IN-2013-71249,10/17/2013,10/18/2013,First Class,CR-12730,Craig Reiter,Brisbane,Queensland,Technology,Phones,"Nokia Smart Phone, with Caller ID",Medium
3,ES-2013-1579342,1/28/2013,1/30/2013,First Class,KM-16375,Katherine Murray,Berlin,Berlin,Technology,Phones,"Motorola Smart Phone, Cordless",Medium
4,SG-2013-4320,11-05-2013,11-06-2013,Same Day,RH-9495,Rick Hansen,Dakar,Dakar,Technology,Copiers,"Sharp Wireless Fax, High-Speed",Critical


In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
le=LabelEncoder()

In [24]:
df_cat_e=df_cat.apply(le.fit_transform)

In [25]:
df_cat_e.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Market,Region,Product ID,Category,Sub-Category,Product Name,Order Priority,pf
0,32,342,343,1,315,296,0,228,141,66,6,6,232,2,0,222,0,363
1,296,10,11,2,199,194,1,331,140,4,0,9,86,0,4,205,0,32
2,290,128,134,0,86,88,0,42,168,4,0,9,377,2,7,203,3,397
3,132,101,108,0,217,201,2,31,22,26,4,3,374,2,7,198,3,52
4,455,148,153,1,317,296,0,73,44,54,1,0,382,2,5,269,0,200


In [26]:
def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)  # First quartile (25th percentile)
        Q3 = df[col].quantile(0.75)  # Third quartile (75th percentile)
        IQR = Q3 - Q1               # Interquartile range
        lower_bound = Q1 - 1.5 * IQR  # Lower bound
        upper_bound = Q3 + 1.5 * IQR  # Upper bound

        # Remove outliers
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    return df

In [27]:
columns_to_check = ['Row ID','Sales','Quantity','Discount','Profit','Shipping Cost']    

df_no_outliers = remove_outliers_iqr(df_num, columns_to_check)

print("Original DataFrame:")
print(df_num.shape)
print("\nDataFrame after Outlier Treatment:")
print(df_no_outliers.shape)

Original DataFrame:
(500, 6)

DataFrame after Outlier Treatment:
(401, 6)


In [29]:
df_no_outliers.shape

(401, 6)

In [30]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [31]:
columns_to_scale =['Row ID','Sales','Quantity','Discount','Profit','Shipping Cost']    


In [32]:
minmax_scaler = MinMaxScaler()
df_minmax_scaled = df_no_outliers.copy()
df_minmax_scaled[columns_to_scale] = minmax_scaler.fit_transform(df_no_outliers[columns_to_scale])


In [33]:
df_minmax_scaled.head()

,Row ID,Sales,Quantity,Discount,Profit,Shipping Cost
24,0.029516,0.243940,0.230769,0.0,0.453796,1.000000
25,0.066880,0.328826,0.538462,0.8,0.493358,0.998676
26,0.588236,0.244225,0.076923,0.0,0.588075,0.992236
27,0.226194,0.537065,0.384615,0.4,0.902656,0.991711
28,0.727228,0.772528,0.538462,0.8,0.413921,0.967322


In [34]:
standard_scaler = StandardScaler()
df_standard_scaled = df_no_outliers.copy()
df_standard_scaled[columns_to_scale] = standard_scaler.fit_transform(df_no_outliers[columns_to_scale])


In [35]:
df_standard_scaled.head()

,Row ID,Sales,Quantity,Discount,Profit,Shipping Cost
24,-1.799711,-0.536345,-0.824934,-0.703917,-0.060620,2.909685
25,-1.652094,-0.136234,0.757834,2.072698,0.138410,2.904358
26,0.407674,-0.535000,-1.616319,-0.703917,0.614905,2.878458
27,-1.022680,0.845294,-0.033550,0.684391,2.197482,2.876346
28,0.956801,1.955145,0.757834,2.072698,-0.261220,2.778257


In [36]:
final_df=pd.concat([df_standard_scaled,df_cat_e],axis=1)

In [37]:
final_df.shape

(500, 24)

In [38]:
final_df.columns

Index(['Row ID', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost',
       'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'Segment', 'City', 'State', 'Country', 'Market',
       'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name',
       'Order Priority', 'pf'],
      dtype='object')

In [39]:
final_df.drop(columns = ['Row ID', 'Order ID','Product ID','Order Date', 'Ship Date','Customer ID'],axis = 1)

,Sales,Quantity,Discount,Profit,Shipping Cost,Ship Mode,Customer Name,Segment,City,State,Country,Market,Region,Category,Sub-Category,Product Name,Order Priority,pf
24,-0.536345,-0.824934,-0.703917,-0.060620,2.909685,0,255,0,143,39,40,5,7,2,7,199,0,264
25,-0.136234,0.757834,2.072698,0.138410,2.904358,0,365,0,291,175,22,5,3,0,10,142,0,291
26,-0.535000,-1.616319,-0.703917,0.614905,2.878458,0,288,1,300,199,60,0,8,0,10,184,0,344
27,0.845294,-0.033550,0.684391,2.197482,2.876346,2,129,0,175,179,26,4,3,1,1,160,0,439
28,1.955145,0.757834,2.072698,-0.261220,2.778257,0,197,2,181,32,66,6,12,2,7,6,1,232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,NaN,NaN,NaN,NaN,NaN,3,106,2,298,59,15,0,8,2,7,201,3,456
481,NaN,NaN,NaN,NaN,NaN,0,271,0,32,159,66,6,6,1,2,103,1,15
491,NaN,NaN,NaN,NaN,NaN,2,57,1,166,166,48,0,4,2,7,253,3,23
495,NaN,NaN,NaN,NaN,NaN,0,138,0,82,143,45,4,3,0,3,87,1,13


In [40]:
df_clean = final_df.dropna()

In [41]:
X = df_clean.drop(columns=['Profit'])  # All columns except 'Profit'
y = df_clean['Profit']                # Target variable

In [42]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [43]:
# Define the hyperparameter grid for Random Forest Regressor
param_grid = {
    'n_estimators': [50, 100, 200,500],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}


In [44]:
# Initialize the Random Forest Regressor
rf = RandomForestRegressor(random_state=42)


In [45]:
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)


In [46]:
# Perform grid search
grid_search.fit(X_train, y_train)

c:\Users\akhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [None, 5, 10, 15],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200, 500]})

In [47]:
# Print the best hyperparameters
print("Best Hyperparameters:")
print(grid_search.best_params_)


Best Hyperparameters:
{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}


In [48]:
# Print the best score
print("Best Score:")
print(grid_search.best_score_)

Best Score:
0.9987811676254195


In [49]:
# Evaluate the best model on the test set
y_pred = grid_search.best_estimator_.predict(X_test)


In [50]:
# Print the mean squared error
print("Mean Squared Error:")
print(mean_squared_error(y_test, y_pred))


Mean Squared Error:
0.002478115683277632


In [51]:
# Print the R-squared score
print("R-squared Score:")
print(r2_score(y_test, y_pred))

R-squared Score:
0.9975772495496118
